In [ ]:
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import sklearn.model_selection
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.utils import shuffle
from google.colab import drive
import numpy as np
import pandas as pd
from copy import deepcopy
import os
import random
import chess
import json
import tqdm
import math
import time

plt.rcParams["figure.figsize"] = (12, 4)
# device = torch.device("cuda:0")
# print(device)

In [ ]:
fit = lambda x: min(1, max(x, -1))
coef = 2000

def get_material(fen):
    
    board = chess.Board(fen)
    
    w, b = chess.WHITE, chess.BLACK
    
    if board.turn == chess.BLACK:
        w, b = b, w
    
    material = 0.0
    material += len(board.pieces(chess.PAWN,   w)) * 1 *  1
    material += len(board.pieces(chess.PAWN,   b)) * 1 * -1
    material += len(board.pieces(chess.KNIGHT, w)) * 3 *  1
    material += len(board.pieces(chess.KNIGHT, b)) * 3 * -1
    material += len(board.pieces(chess.BISHOP, w)) * 3 *  1
    material += len(board.pieces(chess.BISHOP, b)) * 3 * -1
    material += len(board.pieces(chess.ROOK,   w)) * 5 *  1
    material += len(board.pieces(chess.ROOK,   b)) * 5 * -1
    material += len(board.pieces(chess.QUEEN,  w)) * 9 *  1
    material += len(board.pieces(chess.QUEEN,  b)) * 9 * -1
    
    return material

def get_scp(fen):
    
    board = chess.Board(fen)
    
    w, b = chess.WHITE, chess.BLACK
    
    if board.turn == chess.BLACK:
        board = board.transform(chess.flip_vertical)
        # board = board.transform(chess.flip_horizontal)
        w, b = b, w

    data = np.zeros((12, 8, 8))
    
    for i in board.pieces(chess.PAWN, w):
        data[0, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.PAWN, b):
        data[1, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.KNIGHT, w):
        data[2, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.KNIGHT, b):
        data[3, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.BISHOP, w):
        data[4, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.BISHOP, b):
        data[5, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.ROOK, w):
        data[6, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.ROOK, b):
        data[7, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.QUEEN, w):
        data[8, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.QUEEN, b):
        data[9, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.KING, w):
        data[10, 7 - i // 8, i % 8] = 1
    
    for i in board.pieces(chess.KING, b):
        data[11, 7 - i // 8, i % 8] = 1
    
    return data

def get_halfkp(fen):

    board = chess.Board(fen)
    
    w, b = chess.WHITE, chess.BLACK
    
    if board.turn == chess.BLACK:
        board = board.transform(chess.flip_vertical)
        #board = board.transform(chess.flip_horizontal)
        w, b = b, w
        
    data = []
    
    for clr in [w, b]:
        for sq1 in range(64):
            for piece in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]:
                for sq2 in range(64):
                    if board.piece_at(sq1) and board.piece_at(sq2) and board.piece_type_at(sq1) is chess.KING and board.piece_type_at(sq2) is piece and board.color_at(sq1) is w and board.color_at(sq2) is clr:
                        data.append(1)
                    else:
                        data.append(0)
    w, b = b, w
    for clr in [w, b]:
        for sq1 in range(64):
            for piece in [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN]:
                for sq2 in range(64):
                    if board.piece_at(sq1) and board.piece_at(sq2) and board.piece_type_at(sq1) is chess.KING and board.piece_type_at(sq2) is piece and board.color_at(sq1) is w and board.color_at(sq2) is clr:
                        data.append(1)
                    else:
                        data.append(0)
    return data

def create_batch(x_df, y_df):
    x, y = [], []
    sz = len(x_df)
    for it in range(sz):
        board = chess.Board(x_df[it])
        z = board.turn * 2 - 1
        x.append(get_scp(x_df[it]))
        cp = 0
        if "#-" in y_df[it]:
            cp = -9000
        elif "#" in y_df[it]:
            cp = 9000
        else:
            try:
                cp = int(y_df[it])
            except:
                cp = 0
        y.append(0.5 + 0.5 * fit(z * cp / coef))
    return torch.tensor(np.array(x)).float(), torch.tensor(np.array(y)).float()

def update_l_r(l, r, sz, n):
    l += sz
    r += sz
    if r >= n - 20000:
        l = 0
        r = sz
    return l, r

In [ ]:
class Simple_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(768, 2048)
        self.lin2 = nn.Linear(2048, 2048)
        self.lin3 = nn.Linear(2048, 2048)
        self.lin4 = nn.Linear(2048, 1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.lin1(x)
        x = torch.relu(x)
        x = self.lin2(x)
        x = torch.relu(x)
        x = self.lin3(x)
        x = torch.relu(x)
        x = self.lin4(x)
        x = torch.clamp(x, max = 1.0, min = 0.0)
        return x

net = Simple_Net()
# net.to(device)
# net = torch.load("/content/1655848175_1929.pt", map_location=torch.device('cpu'))
# net.eval()

# net = torch.load("/content/1655415573_849.pt")
# net.eval()

optimizer = torch.optim.Adam(net.parameters(), lr = 0.7 * 1e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.7)
mse = nn.MSELoss()
mae = nn.L1Loss()

In [ ]:
drive.mount('/content/drive')

# df = pd.read_csv("/content/drive/MyDrive/chessData.csv")
df = pd.read_csv("/content/drive/MyDrive/random_evals.csv")
# df3 = pd.read_csv("/content/drive/MyDrive/tactic_evals.csv")
# df = pd.read_csv("/content/drive/MyDrive/dataset.csv")

print(len(df))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1000273


In [ ]:
# df = pd.concat([df1, df2], ignore_index=True, sort=False)
df = shuffle(df, random_state = 42)
df.reset_index(inplace=True, drop=True)
# df = df[:1000000]

In [ ]:
n = len(df)
print(n)
batch_size = 256
iters = (n - 20000) // batch_size
epoches = 100

1000273


In [ ]:
test, train = [], []
best = 999
last_time = time.time()

l, r = 0, batch_size
batch_x, batch_y = [], []

test_x, test_y = create_batch(list(df['FEN'][-20000:]), list(df['Evaluation'][-20000:]))

old_name = ""

In [ ]:
for epoch in range(epoches):      
    for it in range(iters):
        batch_x, batch_y = create_batch(list(df['FEN'][l:r]), list(df['Evaluation'][l:r]))
        l, r = update_l_r(l, r, batch_size, n)
        error = mse(net(batch_x).view(batch_y.shape), batch_y)
        optimizer.zero_grad()
        error.backward()
        optimizer.step()
        train.append(error.item())
        
        error = mae(net(test_x).view(test_y.shape), test_y)
        test.append(coef * error.item() / 50)
        
        clear_output(wait = True)
        plt.subplot(121)
        plt.plot(list(range(len(train))), train, label=u"training kurva")
        plt.xlabel(u"iter")
        plt.ylabel(u"loss")
        plt.legend()
        plt.grid()
        
        plt.subplot(122)
        plt.plot(list(range(len(test))), test, label=u"testing kurva")
        plt.xlabel(u"iter")
        plt.ylabel(u"loss")
        plt.legend()
        plt.grid()

        plt.show()
        print("epoch: {}\t iter: {}\t loss: {}\t lr: {}\t {}s".format(epoch, it, round(test[-1], 6), round(optimizer.param_groups[0]['lr'], 6), round(time.time() - last_time, 3)))
        last_time = time.time()
        
        if test[-1] < best:
            if os.path.exists(old_name):
                os.remove(old_name)
            old_name = str(int(time.time())) + "_" + str(int(test[-1] * 1000)) + ".pt"
            torch.save(net, old_name)
            if int(10 * test[-1]) - int(10 * best) > 1:
                torch.save(net, "/content/drive/MyDrive/" + oldname)
            best = test[-1]
            
    l = 0 
    r = batch_size
    scheduler.step()
    